In [ ]:
'''
This code updates the asset data

'''

import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/test/asset_emissions_country_subsector.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()


print("Getting max month...")
max_date = con.execute(f"""
    select max(start_time)
    from postgres_scan('{postgres_url}', 'public', 'asset_emissions')                       
""").fetchone()[0]

print("Running asset-level query and writing to parquet file, this may take a while...")
con.execute(f"""
    INSTALL postgres;
    LOAD postgres;

    CREATE TABLE asset_emissions_parquet AS
    SELECT ae.iso3_country,
        ae.original_inventory_sector,
        ae.start_time,
        ae.gas,
        sch.sector,
        ca.name as country_name,
        ca.continent,
        ca.unfccc_annex,
        ca.em_finance,
        ca.eu,
        ca.oecd,
        ca.developed_un,
        ae.release,
        sum(emissions_quantity) emissions_quantity,
        sum(activity) activity,
        sum(emissions_quantity) / sum(activity) weighted_average_emissions_factor
    
    FROM postgres_scan('{postgres_url}', 'public', 'asset_emissions') ae
    LEFT JOIN postgres_scan('{postgres_url}', 'public', 'country_analysis') ca
        ON CAST(ca.iso3_country AS VARCHAR) = CAST(ae.iso3_country AS VARCHAR)
    LEFT JOIN (
        SELECT DISTINCT sector, subsector FROM postgres_scan('{postgres_url}', 'public', 'asset_schema')
    ) sch
        ON CAST(sch.subsector AS VARCHAR) = CAST(ae.original_inventory_sector AS VARCHAR)
    
    WHERE ae.start_time >= (DATE '{max_date}' - INTERVAL '36 months')
      AND ae.gas = 'co2e_100yr'
      AND ae.most_granular = TRUE
    
    GROUP BY ae.iso3_country,
        ae.original_inventory_sector,
        ae.start_time,
        ae.gas,
        sch.sector,
        ca.name,
        ca.continent,
        ca.unfccc_annex,
        ca.em_finance,
        ca.eu,
        ca.oecd,
        ca.developed_un,
        ae.release;

    COPY asset_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
""")
con.close()

print("✅ Asset parquet file exported")

In [ ]:
import sys
print(sys.executable)



In [ ]:
'''
This is a script to take raw csvs in data/raw_csvs folder and covert them to 
parquets for manageable GitHub storage and limited memory usage (DuckDB).
'''

import pandas as pd
from pathlib import Path

# Set input and output directories
input_dir = Path("data/raw_csvs")
output_dir = Path("data")

# Make sure the output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Loop through all CSV files in the input directory
for csv_file in input_dir.glob("*.csv"):
    print(f"Converting {csv_file.name}...")

    # Read CSV into DataFrame
    df = pd.read_csv(csv_file)

    # Create output path by replacing .csv with .parquet
    parquet_file = output_dir / csv_file.with_suffix(".parquet").name

    # Write to Parquet
    df.to_parquet(parquet_file, engine="pyarrow", index=False)
    print(f"Saved to {parquet_file}")

     # Delete original CSV
    csv_file.unlink()
    print(f"Deleted original CSV: {csv_file.name}")

print("✅ CSV to Parquet conversion complete.")

In [ ]:
# map_region_condition(region_selection)

test = map_region_condition('Asia')

print(test)
print(test['column_name'])
print(test['column_value'])

In [ ]:
import duckdb

con = duckdb.connect()

# Replace with your actual path if needed
parquet_path = "data/country_subsector_emissions_totals.parquet"

# Count how many rows you actually wrote
result = con.execute(f"""SELECT sum(emissions_quantity) FROM '{parquet_path}' where start_time = '2025-02-01' and original_inventory_sector not in ('forest-land-clearing',
                                                'forest-land-degradation',
                                                'forest-land-fires',
                                                'net-forest-land',
                                                'net-shrubgrass',
                                                'net-wetland',
                                                'removals',
                                                'shrubgrass-fires',
                                                'water-reservoirs',
                                                'wetland-fires')""").df()


print(result)

In [ ]:
import duckdb

con = duckdb.connect()

# parquet_path = "data/country_subsector_emissions_totals_202504.parquet"
parquet_path = "data/asset_emissions_country_subsector.parquet"

result = con.execute(f""" 
                     SELECT DISTINCT original_inventory_sector
      FROM '{parquet_path}'
      WHERE gas = 'co2e_100yr'
         AND iso3_country = 'USA'
"""
).df()

print(result)

In [ ]:
import duckdb

con = duckdb.connect()

result = con.execute(f"""
                         SELECT 
        strftime(start_time, '%Y-%m') AS year_month,
        SUM(activity) AS activity,
        SUM(emissions_quantity) AS emissions_quantity
    FROM 'data/test/asset_emissions_country_subsector.parquet'
    WHERE gas = 'co2e_100yr' -- AND original_inventory_sector = 'coal-mining'
        and original_inventory_sector not in ('forest-land-clearing',
                                                'forest-land-degradation',
                                                'forest-land-fires',
                                                'net-forest-land',
                                                'net-shrubgrass',
                                                'net-wetland',
                                                'removals',
                                                'shrubgrass-fires',
                                                'water-reservoirs',
                                                'wetland-fires')
    GROUP BY year_month
    ORDER BY year_month
                     """).df()

print(result)

In [ ]:
!pip install -r requirements.txt

In [ ]:
result.df()

In [ ]:

# ------------------------------------ GADM 1 Emissions ------------------------------------

import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/emissions_reduction/gadm_1_emissions.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()


print('Running query')
con.execute(f'''
    INSTALL postgres;
    LOAD postgres;

    CREATE TABLE gadm_1_emissions_parquet AS
    select extract(year from g1e.start_time) as year 
        , g1e.gadm_id
        , gb.admin_level
        , g1e.iso3_country
        , ca.name as country_name
        , gb.name gadm_1_name
        , gb.corrected_name gadm_1_corrected_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , g1e.original_inventory_sector subsector
        , g1e.gas
        , sum(asset_activity) asset_activity
        , sum(asset_emissions) asset_emissions
        , sum(remainder_activity) remainder_activity
        , sum(remainder_emissions) remainder_emissions
        , sum(asset_emissions) + sum(remainder_emissions) as emissions_quantity

    from postgres_scan('{postgres_url}', 'public', 'gadm_1_emissions') g1e
    inner join (
        select distinct gadm_id
            , name
            , corrected_name
            , admin_level
        from postgres_scan('{postgres_url}','public', 'gadm_boundaries') 
        where admin_level = 1
    ) as gb
        on g1e.gadm_id = gb.gadm_id
    left join (
        select distinct sector
            , subsector
        from postgres_scan('{postgres_url}','public', 'asset_schema') 
    ) asch
        on cast(asch.subsector as varchar) = cast(g1e.original_inventory_sector as varchar)
    left join postgres_scan('{postgres_url}','public', 'country_analysis') ca
		on cast(ca.iso3_country as varchar) = cast(g1e.iso3_country as varchar)

    where g1e.gas = 'co2e_100yr'
        and extract(year from start_time) = 2024
        and g1e.original_inventory_sector not in ('forest-land-clearing',
                                                'forest-land-degradation',
                                                'forest-land-fires',
                                                'net-forest-land',
                                                'net-shrubgrass',
                                                'net-wetland',
                                                'removals',
                                                'shrubgrass-fires',
                                                'water-reservoirs',
                                                'wetland-fires')

    group by extract(year from g1e.start_time) 
        , g1e.gadm_id
        , gb.admin_level
        , g1e.iso3_country
        , ca.name
        , gb.name 
        , gb.corrected_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , g1e.original_inventory_sector
        , g1e.gas;

    COPY gadm_1_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
''')
con.close()

print('Complete')

In [ ]:

# ------------------------------------ GADM 2 Emissions ------------------------------------

import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/emissions_reduction/gadm_2_emissions.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()


print('Running query')
con.execute(f'''
    INSTALL postgres;
    LOAD postgres;

    CREATE TABLE gadm_2_emissions_parquet AS
    select extract(year from ge.start_time) as year 
        , gb1.gadm_id gadm_1_id
        , gb1.name gadm_1_name
        , gb1.corrected_name gadm_1_corrected_name
        , ge.gadm_id gadm_2_id
        , gb2.name gadm_2_name
        , gb2.corrected_name gadm_2_corrected_name
        , gb2.admin_level
        , ge.iso3_country
        , ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , ge.original_inventory_sector
        , sum(asset_activity) asset_activity
        , sum(asset_emissions) asset_emissions
        , sum(remainder_activity) remainder_activity
        , sum(remainder_emissions) remainder_emissions
        , sum(asset_emissions) + sum(remainder_emissions) as emissions_quantity

    from postgres_scan('{postgres_url}','public', 'gadm_emissions') ge
    inner join (
        select distinct gadm_id
            , immediate_parent
            , name
            , corrected_name
            , admin_level
        from postgres_scan('{postgres_url}','public', 'gadm_boundaries')
        where admin_level = 2
    ) as gb2
        on ge.gadm_id = gb2.gadm_id
    left join (
        select distinct sector
            , subsector
        from postgres_scan('{postgres_url}','public', 'asset_schema')
    ) asch
        on cast(asch.subsector as varchar) = cast(ge.original_inventory_sector as varchar)
    left join (
        select gadm_id
            , name
            , corrected_name
        from postgres_scan('{postgres_url}','public', 'gadm_boundaries')
        where admin_level = 1
    ) gb1
        on gb1.gadm_id = gb2.immediate_parent
    left join postgres_scan('{postgres_url}','public', 'country_analysis') ca
        on cast(ca.iso3_country as varchar) = cast(ge.iso3_country as varchar)

    where ge.gas = 'co2e_100yr'
        and extract(year from start_time) = 2024
        and ge.original_inventory_sector not in ('forest-land-clearing',
                                                'forest-land-degradation',
                                                'forest-land-fires',
                                                'net-forest-land',
                                                'net-shrubgrass',
                                                'net-wetland',
                                                'removals',
                                                'shrubgrass-fires',
                                                'water-reservoirs',
                                                'wetland-fires')

    group by extract(year from ge.start_time)
        , gb1.gadm_id 
        , gb1.name
        , gb1.corrected_name
        , ge.gadm_id 
        , gb2.name
        , gb2.corrected_name
        , gb2.admin_level
        , ge.iso3_country
        , ca.name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , ge.original_inventory_sector;

    COPY gadm_2_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
''')
con.close()

print('Complete')

In [ ]:
# ------------------------------------ Asset Annual Emissions ------------------------------------

import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/emissions_reduction/asset_annual_emissions.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()

print('Running query...')
con.execute( f'''
	INSTALL postgres;
	LOAD postgres;

	CREATE TABLE asset_annual_emissions_parquet AS
	select extract(year from ae.start_time
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector as subsector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units
		, sum(capacity) capacity
		, sum(activity) activity
		, avg(emissions_factor) average_emissions_factor
		, sum(emissions_quantity) emissions_quantity

	from postgres_scan('{postgres_url}','public', 'asset_emissions') ae
	left join postgres_scan('{postgres_url}','public', 'asset_information') ai
		on ai.asset_id = ae.asset_id
	left join postgres_scan('{postgres_url}','public', 'asset_location') al
		on al.asset_id = ae.asset_id
	left join (
		select distinct sector, subsector from postgres_scan('{postgres_url}','public', 'asset_schema')
	) asch
		on cast(asch.subsector as varchar) = cast(ae.original_inventory_sector as varchar)
	left join postgres_scan('{postgres_url}','public', 'country_analysis') ca
		on cast(ca.iso3_country as varchar) = cast(ae.iso3_country as varchar)

	where extract(year from ae.start_time) = 2024
		and ae.most_granular = true
		and ae.gas = 'co2e_100yr'
		and ae.original_inventory_sector not in ('forest-land-clearing',
													'forest-land-degradation',
													'forest-land-fires',
													'net-forest-land',
													'net-shrubgrass',
													'net-wetland',
													'removals',
													'shrubgrass-fires',
													'water-reservoirs',
													'wetland-fires')

	group by extract(year from ae.start_time)
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units;
            
    COPY asset_annual_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
            
    ''')

con.close()

print('Complete')

In [ ]:
# ------------------------------------ Asset MONTHLY Emissions ------------------------------------

import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/emissions_reduction/asset_monthly_emissions.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()

print('Running query...')
con.execute( f'''
	INSTALL postgres;
	LOAD postgres;

	CREATE TABLE asset_annual_emissions_parquet AS
	select ae.start_time
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector as subsector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units
		, sum(capacity) capacity
		, sum(activity) activity
		, avg(emissions_factor) average_emissions_factor
		, sum(emissions_quantity) emissions_quantity

	from postgres_scan('{postgres_url}','public', 'asset_emissions') ae
	left join postgres_scan('{postgres_url}','public', 'asset_information') ai
		on ai.asset_id = ae.asset_id
	left join postgres_scan('{postgres_url}','public', 'asset_location') al
		on al.asset_id = ae.asset_id
	left join (
		select distinct sector, subsector from postgres_scan('{postgres_url}','public', 'asset_schema')
	) asch
		on cast(asch.subsector as varchar) = cast(ae.original_inventory_sector as varchar)
	left join postgres_scan('{postgres_url}','public', 'country_analysis') ca
		on cast(ca.iso3_country as varchar) = cast(ae.iso3_country as varchar)

	where extract(year from ae.start_time) = 2024
		and ae.most_granular = true
		and ae.gas = 'co2e_100yr'
		and ae.original_inventory_sector not in ('forest-land-clearing',
													'forest-land-degradation',
													'forest-land-fires',
													'net-forest-land',
													'net-shrubgrass',
													'net-wetland',
													'removals',
													'shrubgrass-fires',
													'water-reservoirs',
													'wetland-fires')

	group by ae.start_time
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units;
            
    COPY asset_monthly_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
            
    ''')

con.close()

print('Complete')

In [ ]:
# ------------------------------------ City Emissions ------------------------------------


import duckdb
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

load_dotenv()

# Build SQLAlchemy engine for PostgreSQL
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

postgres_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
parquet_path = "data/emissions_reduction/city_emissions.parquet"

# Use DuckDB to write directly from PostgreSQL to Parquet
con = duckdb.connect()

print('Running query...')
con.execute( f'''
	INSTALL postgres;
	LOAD postgres;

	CREATE TABLE city_emissions_parquet AS
    
	select extract(year from start_time) as year
		, ce.city_id
		, cb.name as city_name
		, cb.corrected_name as corrected_name
		, ce.iso3_country
		, ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ce.original_inventory_sector as subsector
		, sum(asset_activity) asset_activity
		, sum(asset_emissions) asset_emissions
		, sum(remainder_activity) remainder_activity
		, sum(remainder_emissions) remainder_emissions
		, sum(asset_emissions) + sum(remainder_emissions) as emissions_quantity

	from postgres_scan('{postgres_url}','public', 'city_emissions') ce
	left join postgres_scan('{postgres_url}','public', 'city_boundaries') cb
		on cb.city_id = ce.city_id
	left join (
		select distinct sector, subsector
		from postgres_scan('{postgres_url}','public', 'asset_schema')
	) asch
		on cast(asch.subsector as varchar) = cast(ce.original_inventory_sector as varchar)
	left join postgres_scan('{postgres_url}','public', 'country_analysis') ca
		on cast(ca.iso3_country as varchar) = cast(ce.iso3_country as varchar)

	where extract(year from ce.start_time) = 2024
		and ce.gas = 'co2e_100yr'
		and ce.original_inventory_sector not in ('forest-land-clearing',
														'forest-land-degradation',
														'forest-land-fires',
														'net-forest-land',
														'net-shrubgrass',
														'net-wetland',
														'removals',
														'shrubgrass-fires',
														'water-reservoirs',
														'wetland-fires')

	group by extract(year from start_time) 
		, ce.city_id
		, cb.name 
		, cb.corrected_name 
		, ce.iso3_country
		, ca.name 
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ce.original_inventory_sector;
            
    COPY city_emissions_parquet TO '{parquet_path}' (FORMAT PARQUET);
            
    ''')

con.close()

print('Complete')

In [ ]:
# --------------------------------------------------------- GADM 2 BATCH -----------------------------------------------------------------

import psycopg2
from urllib.parse import quote_plus
import pyarrow as pa
import pyarrow.parquet as pq
import csv
import os

user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor(name='parquet_cursor')  # server-side cursor


cur.execute("""
     select extract(year from ge.start_time) as year 
        , gb1.gadm_id gadm_1_id
        , gb1.name gadm_1_name
        , gb1.corrected_name gadm_1_corrected_name
        , ge.gadm_id gadm_2_id
        , gb2.name gadm_2_name
        , gb2.corrected_name gadm_2_corrected_name
        , gb2.admin_level
        , ge.iso3_country
        , ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , ge.original_inventory_sector
        , sum(asset_activity) asset_activity
        , sum(asset_emissions) asset_emissions
        , sum(remainder_activity) remainder_activity
        , sum(remainder_emissions) remainder_emissions
        , sum(asset_emissions) + sum(remainder_emissions) as emissions_quantity

    from gadm_emissions ge
    inner join (
        select distinct gadm_id
            , immediate_parent
            , name
            , corrected_name
            , admin_level
        from gadm_boundaries
        where admin_level = 2
    ) as gb2
        on ge.gadm_id = gb2.gadm_id
    left join (
        select distinct sector
            , subsector
        from asset_schema
    ) asch
        on cast(asch.subsector as varchar) = cast(ge.original_inventory_sector as varchar)
    left join (
        select gadm_id
            , name
            , corrected_name
        from gadm_boundaries
        where admin_level = 1
    ) gb1
        on gb1.gadm_id = gb2.immediate_parent
    left join country_analysis ca
        on cast(ca.iso3_country as varchar) = cast(ge.iso3_country as varchar)

    where ge.gas = 'co2e_100yr'
        and extract(year from start_time) = 2024
        and ge.original_inventory_sector not in ('forest-land-clearing',
                                                'forest-land-degradation',
                                                'forest-land-fires',
                                                'net-forest-land',
                                                'net-shrubgrass',
                                                'net-wetland',
                                                'removals',
                                                'shrubgrass-fires',
                                                'water-reservoirs',
                                                'wetland-fires')

    group by extract(year from ge.start_time)
        , gb1.gadm_id 
        , gb1.name
        , gb1.corrected_name
        , ge.gadm_id 
        , gb2.name
        , gb2.corrected_name
        , gb2.admin_level
        , ge.iso3_country
        , ca.name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
        , asch.sector
        , ge.original_inventory_sector
    """)

# Set up Parquet writer
batch_size = 10000
output_file = "data/emissions_reduction/gadm_2_emissions.parquet"
batch_count = 0
total_rows = 0

print("executing gadm_2 query...")

# Fetch first batch
rows = cur.fetchmany(batch_size)
if not rows:
    raise Exception("No data returned from query.")

field_names = [desc[0] for desc in cur.description]
first_table = pa.Table.from_pylist([dict(zip(field_names, row)) for row in rows])
writer = pq.ParquetWriter(output_file, first_table.schema)
writer.write_table(first_table)
batch_count += 1
total_rows += len(rows)
print(f"Processed batch {batch_count} ({len(rows)} rows), total rows: {total_rows}")

# Process remaining batches
while True:
    rows = cur.fetchmany(batch_size)
    if not rows:
        break

    table = pa.Table.from_pylist([dict(zip(field_names, row)) for row in rows])
    table = table.cast(writer.schema)  # ensure schema matches first batch
    writer.write_table(table)

    batch_count += 1
    total_rows += len(rows)
    print(f"Processed batch {batch_count} ({len(rows)} rows), total rows: {total_rows}")

writer.close()
cur.close()
conn.close()
print("Export complete.")

In [ ]:
# --------------------------------------------------------- CITY BATCH -----------------------------------------------------------------

import psycopg2
from urllib.parse import quote_plus
import pyarrow as pa
import pyarrow.parquet as pq
import os

user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor(name='parquet_cursor')  # server-side cursor

cur.execute("""
    SELECT extract(year from start_time) AS year,
           ce.city_id,
           cb.name AS city_name,
           cb.corrected_name AS corrected_name,
           ce.iso3_country,
           ca.name AS country_name,
           ca.continent,
           ca.eu,
           ca.oecd,
           ca.unfccc_annex,
           ca.developed_un,
           ca.em_finance,
           asch.sector,
           ce.original_inventory_sector AS subsector,
           SUM(asset_activity) AS asset_activity,
           SUM(asset_emissions) AS asset_emissions,
           SUM(remainder_activity) AS remainder_activity,
           SUM(remainder_emissions) AS remainder_emissions,
           SUM(asset_emissions) + SUM(remainder_emissions) AS emissions_quantity
    FROM city_emissions ce
    LEFT JOIN city_boundaries cb ON cb.city_id = ce.city_id
    LEFT JOIN (
        SELECT DISTINCT sector, subsector FROM asset_schema
    ) asch ON CAST(asch.subsector AS varchar) = CAST(ce.original_inventory_sector AS varchar)
    LEFT JOIN country_analysis ca ON CAST(ca.iso3_country AS varchar) = CAST(ce.iso3_country AS varchar)
    WHERE extract(year FROM ce.start_time) = 2024
      AND ce.gas = 'co2e_100yr'
      AND ce.original_inventory_sector NOT IN (
          'forest-land-clearing', 'forest-land-degradation', 'forest-land-fires',
          'net-forest-land', 'net-shrubgrass', 'net-wetland', 'removals',
          'shrubgrass-fires', 'water-reservoirs', 'wetland-fires'
      )
    GROUP BY extract(year FROM start_time),
             ce.city_id, cb.name, cb.corrected_name,
             ce.iso3_country, ca.name, ca.continent, ca.eu, ca.oecd,
             ca.unfccc_annex, ca.developed_un, ca.em_finance,
             asch.sector, ce.original_inventory_sector
""")

# Set up Parquet writer
batch_size = 10000
output_file = "data/emissions_reduction/city_emissions.parquet"
batch_count = 0
total_rows = 0

print("executing city query...")

# Fetch first batch
rows = cur.fetchmany(batch_size)
if not rows:
    raise Exception("No data returned from query.")

field_names = [desc[0] for desc in cur.description]
first_table = pa.Table.from_pylist([dict(zip(field_names, row)) for row in rows])
writer = pq.ParquetWriter(output_file, first_table.schema)
writer.write_table(first_table)
batch_count += 1
total_rows += len(rows)
print(f"Processed batch {batch_count} ({len(rows)} rows), total rows: {total_rows}")

# Process remaining batches
while True:
    rows = cur.fetchmany(batch_size)
    if not rows:
        break

    table = pa.Table.from_pylist([dict(zip(field_names, row)) for row in rows])
    table = table.cast(writer.schema)  # ensure schema matches first batch
    writer.write_table(table)

    batch_count += 1
    total_rows += len(rows)
    print(f"Processed batch {batch_count} ({len(rows)} rows), total rows: {total_rows}")

writer.close()
cur.close()
conn.close()
print("Export complete.")


In [ ]:
# --------------------------------------------------------- ASSET BATCH -----------------------------------------------------------------

import psycopg2
from urllib.parse import quote_plus
import pyarrow as pa
import pyarrow.parquet as pq
import os

# Get credentials
user = quote_plus(os.getenv("CLIMATETRACE_USER"))
password = quote_plus(os.getenv("CLIMATETRACE_PASS"))
host = os.getenv("CLIMATETRACE_HOST")
port = os.getenv("CLIMATETRACE_PORT")
database = os.getenv("CLIMATETRACE_DB")

# Connect to Postgres
conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor(name='parquet_cursor')  # server-side cursor
cur.itersize = 10000  # tuning parameter

# Execute query
print("Executing asset query...")

cur.execute("""
    select extract(year from ae.start_time) as year
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name as country_name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector as subsector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units
		, sum(capacity) capacity
		, sum(activity) activity
		, avg(emissions_factor) average_emissions_factor
		, sum(emissions_quantity) emissions_quantity

	from asset_emissions ae
	left join asset_information ai
		on ai.asset_id = ae.asset_id
	left join asset_location al
		on al.asset_id = ae.asset_id
	left join (
		select distinct sector, subsector from asset_schema
	) asch
		on cast(asch.subsector as varchar) = cast(ae.original_inventory_sector as varchar)
	left join country_analysis ca
		on cast(ca.iso3_country as varchar) = cast(ae.iso3_country as varchar)

	where extract(year from ae.start_time) = 2024
		and ae.most_granular = true
		and ae.gas = 'co2e_100yr'
		and ae.original_inventory_sector not in ('forest-land-clearing',
													'forest-land-degradation',
													'forest-land-fires',
													'net-forest-land',
													'net-shrubgrass',
													'net-wetland',
													'removals',
													'shrubgrass-fires',
													'water-reservoirs',
													'wetland-fires')

	group by extract(year from ae.start_time)
		, ae.asset_id
		, ai.asset_type
		, ai.asset_name
		, ae.iso3_country
		, ca.name
        , ca.continent
        , ca.eu
        , ca.oecd
        , ca.unfccc_annex
        , ca.developed_un
        , ca.em_finance
		, asch.sector
		, ae.original_inventory_sector
		, al.gadm_1
		, al.gadm_2
		, al.ghs_fua
		, al.city_id
		, ae.other1
		, ae.other2
		, ae.other3
		, ae.other4
		, ae.other5
		, ae.other6
		, ae.other7
		, ae.other8
		, ae.other9
		, ae.other10
		, ae.activity_units
""")

schema = pa.schema([
    ("year", pa.int32()),
    ("asset_id", pa.string()),
    ("asset_type", pa.string()),
    ("asset_name", pa.string()),
    ("iso3_country", pa.string()),
    ("country_name", pa.string()),
    ("continent", pa.string()),
    ("eu", pa.bool_()),
    ("oecd", pa.bool_()),
    ("unfccc_annex", pa.bool()),
    ("developed_un", pa.bool_()),
    ("em_finance", pa.bool()),
    ("sector", pa.string()),
    ("subsector", pa.string()),
    ("gadm_1", pa.string()),
    ("gadm_2", pa.string()),
    ("ghs_fua", pa.string()),
    ("city_id", pa.string()),
    ("other1", pa.string()),
    ("other2", pa.string()),
    ("other3", pa.string()),
    ("other4", pa.string()),
    ("other5", pa.string()),
    ("other6", pa.string()),
    ("other7", pa.string()),
    ("other8", pa.string()),
    ("other9", pa.string()),
    ("other10", pa.string()),
    ("activity_units", pa.string()),
    ("capacity", pa.float64()),
    ("activity", pa.float64()),
    ("average_emissions_factor", pa.float64()),
    ("emissions_quantity", pa.float64()),
])

# --- Output ---
output_file = "data/emissions_reduction/asset_annual_emissions.parquet"
batch_size = 10000
batch_count = 0
total_rows = 0

print("Executing query and writing batches...")

writer = None
field_names = [desc[0] for desc in cur.description]

while True:
    rows = cur.fetchmany(batch_size)
    if not rows:
        break

    # Fill missing fields to match schema
    clean_rows = []
    for row in rows:
        row_dict = dict(zip(field_names, row))
        for field in schema.names:
            row_dict.setdefault(field, None)
        clean_rows.append(row_dict)

    table = pa.Table.from_pylist(clean_rows, schema=schema)

    if writer is None:
        writer = pq.ParquetWriter(output_file, schema)

    writer.write_table(table)
    batch_count += 1
    total_rows += len(rows)
    print(f"Processed batch {batch_count} ({len(rows)} rows), total rows: {total_rows}")

# --- Close ---
if writer:
    writer.close()
cur.close()
conn.close()
print("Export complete.")

In [ ]:
import duckdb

con = duckdb.connect()

parquet_path = "data/moer/asset_moer_2023.parquet"

asset_moers = con.execute (f"""
    create table asset_moer_parquet as
    select asset_id
        , original_inventory_sector
        , year
        , avg(moer_avg) as moer_avg
    
    from 'data/moer/ct_wt_monthly_moers_2023.csv'
    
    group by asset_id
        , original_inventory_sector
        , year;
                                             
    COPY asset_moer_parquet TO '{parquet_path}' (FORMAT PARQUET);
""")

con.close()


In [ ]:
#62756
import duckdb

con = duckdb.connect()

parquet_path = "data/moer/asset_moer_2023.parquet"

result = con.execute(f"""
    select count(distinct asset_id)
    from '{parquet_path}'  
""").df()

print(result)

In [ ]:
## ---------------------------------- ADD THIS TO ASSET ANNUAL DATA REFRESH --------------------------------------

# import duckdb
from utils.utils import data_add_moer
import pandas as pd

asset_parquet_path = 'data/emissions_reduction/asset_annual_emissions.parquet'
output_path = 'data/emissions_reduction/asset_annual_emissions_moer.parquet'

df_asset = pd.read_parquet(asset_parquet_path)

asset_moer_df = data_add_moer(df_asset, cond={"moer": True})

asset_moer_df.to_parquet(output_path, index=False)


   count_star()
0       2352158


In [ ]:
import duckdb

con = duckdb.connect()

result = con.execute(f"""
    select distinct sector
    from 'data/asset_emissions_country_subsector.parquet'
    """
).df()

print(result)

                          column_name column_type null   key default extra
0                        iso3_country     VARCHAR  YES  None    None  None
1           original_inventory_sector     VARCHAR  YES  None    None  None
2                          start_time   TIMESTAMP  YES  None    None  None
3                                 gas     VARCHAR  YES  None    None  None
4                              sector     VARCHAR  YES  None    None  None
5                        country_name     VARCHAR  YES  None    None  None
6                           continent     VARCHAR  YES  None    None  None
7                        unfccc_annex     BOOLEAN  YES  None    None  None
8                          em_finance     BOOLEAN  YES  None    None  None
9                                  eu     BOOLEAN  YES  None    None  None
10                               oecd     BOOLEAN  YES  None    None  None
11                       developed_un     BOOLEAN  YES  None    None  None
12                       

In [ ]:
# ------------------------------------ SPLITS LARGE ASSET FILE INTO ~50MB CHUNKS ---------------------------------


import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

# === CONFIG ===
input_file = "data/emissions_reduction/asset_annual_emissions.parquet"  # Your large file
output_dir = "data/asset_annual_emissions"  # Destination folder
target_size_mb = 50  # Keep each file safely under 100MB
os.makedirs(output_dir, exist_ok=True)

# Load full Parquet into DataFrame
df = pd.read_parquet(input_file)
total_rows = len(df)

# Estimate file size per row using a small sample
test_sample = df.iloc[:10000]
test_table = pa.Table.from_pandas(test_sample)
pq.write_table(test_table, "temp.parquet")
bytes_per_row = os.path.getsize("temp.parquet") / len(test_sample)
os.remove("temp.parquet")

# Determine number of rows per ~50MB chunk
target_bytes = target_size_mb * 1024 * 1024
rows_per_chunk = int(target_bytes / bytes_per_row)

# Split and write files
for i, start in enumerate(range(0, total_rows, rows_per_chunk)):
    end = min(start + rows_per_chunk, total_rows)
    chunk_df = df.iloc[start:end]
    chunk_table = pa.Table.from_pandas(chunk_df)
    output_path = os.path.join(output_dir, f"chunk_{i+1}.parquet")
    pq.write_table(chunk_table, output_path)
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"Saved {output_path} ({size_mb:.1f} MB, rows {start}–{end})")


Saved data/asset_annual/chunk_1.parquet (53.1 MB, rows 0–585872)
Saved data/asset_annual/chunk_2.parquet (52.9 MB, rows 585872–1171744)
Saved data/asset_annual/chunk_3.parquet (53.1 MB, rows 1171744–1757616)
Saved data/asset_annual/chunk_4.parquet (53.0 MB, rows 1757616–2343488)
Saved data/asset_annual/chunk_5.parquet (14.3 MB, rows 2343488–2485152)


In [19]:
import duckdb

con = duckdb.connect()
asset_annual_path = 'data/asset_annual_emissions/chunk_*.parquet'

test = con.execute(f"""
                   select count(*)
                   from (
                    select asset_id 
                        , iso3_country
                        , country_name
                        , sector
                        , subsector
                        , avg(ef_12_moer) ef_12_moer
                        , sum(activity) activity
                        , sum(emissions_quantity) emissions_quantity
                    
                    from '{asset_annual_path}'

                    group by asset_id
                        , iso3_country
                        , country_name
                        , sector
                        , subsector
                   )
                """).df()

print(test)

   count_star()
0        686460


In [24]:
import duckdb

con = duckdb.connect()

city = con.execute(f"""select * from 'data/emissions_reduction/city_emissions.parquet' where city_name = 'Chicago'""").df()

print(city)

      year       city_id city_name      corrected_name iso3_country  \
0   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
1   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
2   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
3   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
4   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
5   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
6   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
7   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
8   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
9   2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
10  2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
11  2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
12  2024.0  ghs-fua_6654   Chicago  Chicago Urban Area          USA   
13  20

In [31]:
import duckdb

con = duckdb.connect()

city = con.execute(f"""select distinct city_id from 'data/asset_annual_emissions/*.parquet' where city_id is not null""").df()

print(city)

Empty DataFrame
Columns: [city_id]
Index: []
